# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [49]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
weather_data = "./WeatherPy.csv"
vacation_df = pd.read_csv(weather_data)
vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.6001,-73.9662,45.27,77,0,2.19,US,1666357101
1,Hobart,-42.8794,147.3294,63.05,87,75,1.14,AU,1666356965
2,Ushuaia,-54.8000,-68.3000,44.26,57,75,9.22,AR,1666357259
3,Rikitea,-23.1203,-134.9692,70.97,75,8,21.39,PF,1666357260
4,Castro,-24.7911,-50.0119,62.24,97,100,11.25,BR,1666357230


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
#Use the Lat and Lng as locations and Humidity as the weight.
#store lat / long data
locations = vacation_df[["Lat", "Lng"]]
humidity = vacation_df["Humidity"].astype(float)

In [31]:
#create a heat map by location
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Create bank symbol layer
top_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]
)

fig = gmaps.figure()
fig.add_layer(bank_layer)

fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:
#Drop rows that have null values
new_vacation_data = vacation_df.dropna()
#set the temerature range of the ideal location
clean_vacation_data = new_vacation_data[(new_vacation_data['Max Temp'] >= 65) 
                                        & (new_vacation_data['Max Temp'] < 80)
                                        & (new_vacation_data['Wind Speed'] < 10)
                                        & (new_vacation_data['Cloudiness'] == 0)]
clean_vacation_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,Nālāgarh,31.0500,76.7167,72.05,58,0,3.67,IN,1666357281
56,Alice Springs,-23.7000,133.8833,69.35,35,0,0.00,AU,1666357290
94,San Quintín,30.4833,-115.9500,65.84,86,0,0.51,MX,1666357312
136,Henderson,36.0397,-114.9819,68.23,35,0,6.91,US,1666357303
147,Arrondissement d'Ajaccio,42.0000,8.9167,75.18,41,0,3.44,FR,1666357143
162,Laguna,38.4210,-121.4238,68.09,75,0,4.61,US,1666357341
203,Lixouri,38.2019,20.4314,75.20,53,0,0.00,GR,1666357363
219,Bela,25.9333,81.9833,77.76,55,0,5.17,IN,1666357370
244,San Patricio,28.0170,-97.5169,67.84,79,0,7.76,US,1666357383
258,Abhā,18.2164,42.5053,70.63,34,0,8.66,SA,1666357305


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = pd.DataFrame()
hotel_df['Hotel Name'] = ""
hotel_df

,Hotel Name


In [53]:
radius = 5000
query = 'hotel'
lat = "-23.7000"
lng = "133.8833"

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "query": query,
    "location": f"{lat},{lng}",
    "radius": radius,
    "rankby": "distance",
    "key": g_key,
}
# run a request using our params dictionary
response = requests.get(base_url, params=params)
response = response.json()


In [54]:
print(json.dumps(response, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
